# Importing Libraries 

In [1]:
# Using Tensorflow 2.0
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

Instructions for updating:
non-resource variables are not supported in the long term


# Importing Dataset

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


# Weights initialisation 

In [4]:
def weights_initialisation(shape):
    random_initialisation_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(random_initialisation_dist)

#  Bias initialisation 

In [5]:
def bias_initialisation(shape):
    bias_initialisation_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(bias_initialisation_vals)

# Function To create Neural Network

In [6]:
## Convolution Layer
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

## Pooling
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

## Creating Convolution Layer
def convolutional_layer(input_x, shape):
    W = weights_initialisation(shape)
    b = bias_initialisation([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = weights_initialisation([input_size, size])
    b = bias_initialisation([size])
    return tf.matmul(input_layer, W) + b

# Creating Placeholder

In [7]:
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [8]:
x_image = tf.reshape(x,[-1,28,28,1])

# First Convolutional Layer

In [9]:
convo_1 = convolutional_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

#  Second Convolutional Layer

In [10]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

# Output Layer

In [11]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

# Regularizing Neural Network 

In [12]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
y_pred = normal_full_layer(full_one_dropout,10)

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



# Optimizing Neural Network

In [20]:
def Optimizer(name):
    print("Optimizer chosen :- "+ name)
    if name=="GradientDescentOptimizer":
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
    if name=="MomentumOptimizer":
        optimizer = tf.train.MomentumOptimizer(learning_rate=0.0001,momentum=0.9)
    if name=="RMSPropOptimizer":
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001,momentum=0.9)
    if name=="AdamOptimizer":
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    train = optimizer.minimize(cross_entropy)
    return train

In [16]:
# optimizer = tf.train.MomentumOptimizer(learning_rate=0.0001,momentum=0.9)
# train = optimizer.minimize(cross_entropy)

# Running Final Model 

In [22]:

steps = 5000
opt = ["GradientDescentOptimizer", "MomentumOptimizer", "RMSPropOptimizer", "AdamOptimizer"]
for opti in opt:
    train = Optimizer(opti)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        start_time = time.time()
        for i in range(steps):
            batch_x , batch_y = mnist.train.next_batch(100)
            sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
            if i%100 == 0:
                # Test the Train Model
                matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
                acc = tf.reduce_mean(tf.cast(matches,tf.float32))
                Accuracy = sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0})
        print('Accuracy is:- ' + str(Accuracy))
        print("Time taken %s mins:- " % ((time.time() - start_time)/60))
        print('\n')

Optimizer chosen :- GradientDescentOptimizer
Accuracy is:- 0.8473
Time taken 2.940947703520457 mins:- 


Optimizer chosen :- MomentumOptimizer
Accuracy is:- 0.9554
Time taken 3.0980087399482725 mins:- 


Optimizer chosen :- RMSPropOptimizer
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[10000,32,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MaxPool (defined at <ipython-input-6-5b127947f9e0>:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[10000,32,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MaxPool (defined at <ipython-input-6-5b127947f9e0>:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Mean_202/_23]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 succes